# Downloading images from the Hubble Legacy Archive
Assuming we have downloaded the gz_candels_table_2_main_release.csv from https://data.galaxyzoo.org/, let's take a look at it


In [2]:
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations
from astropy import units as u
from astropy import coordinates as coords
import os
from astropy.nddata.utils import Cutout2D
import shutil
from astropy import wcs
from astropy.coordinates import SkyCoord,FK5
from astropy.wcs.utils import skycoord_to_pixel

In [3]:
GZ_catalog = fits.open('schawinski_GZ_2010_catalogue.fits')
catalog = Table(GZ_catalog[1].data)

In [4]:
ObjID = catalog['OBJID'][0]
Morphology = catalog['GZ1_MORPHOLOGY'][0]
BPT_class = catalog['BPT_CLASS'][0]
RA = catalog['RA'][0]
DEC = catalog['DEC'][0]
print(ObjID)

['587722982288785725' '587722982280069271' '587722981748048047' ...
 '588848899924361312' '588298664654930067' '588298664651718676']


In [5]:
obsids = {}  # {obs_id, [ra,dec]}
for gal_ct in range(20):
    # Obtain information for given galaxy
    id_ = ObjID[gal_ct]
    morph_ = Morphology[gal_ct]
    bpt_ = BPT_class[gal_ct]
    ra_ = RA[gal_ct]
    dec_ = DEC[gal_ct]
    # Convert coordinates to use astropy skycoordinates
    pos = coords.SkyCoord(ra=ra_*u.degree, dec= dec_*u.degree, frame='fk5')
    # Use the MAST catalog to query for Hubble observation
    obs = Observations.query_criteria(project='HST',
                                       filters='F160W',
                                       instrument_name='WFC3/IR',
                                       dataproduct_type=["image"],
                                       coordinates=pos)
    
    if obs:
        print("RA:%s DEC:%s"%(ra_,dec_))
        for ob in obs:
            obsids[ob['obs_id']] = [ra_, dec_]
            print('    '+ob['obsid']+' - '+ob['filters'])
            data_products = Observations.get_product_list(ob['obsid'])
            manifest = Observations.download_products(data_products, download_dir='HST/'+ob['obsid']+'-'+ob['filters'], productType='SCIENCE', extension='fits', productSubGroupDescription='DRZ')

RA:187.24380843 DEC:-0.71369079
    2008146298 - F160W
    2008146304 - F160W


RA:224.51223292 DEC:-0.29179222
    2007927826 - F160W
INFO: Found cached file HST/2007927826-F160W/mastDownload/HST/idpw01010/idpw01010_drz.fits with expected size 12732480. [astroquery.query]


In [6]:
for root, dirs, files in os.walk("HST"):
    path = root.split(os.sep)
    #print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        if file.endswith('.fits'):
            print(os.path.join(root, file))
            shutil.copy(os.path.join(root, file), os.path.join('HST-Final', file))

HST/2008146298-F160W/mastDownload/HST/ie4705010/ie4705010_drz.fits
HST/2007927826-F160W/mastDownload/HST/idpw01010/idpw01010_drz.fits
HST/2008146304-F160W/mastDownload/HST/ie4705020/ie4705020_drz.fits


In [10]:
pix=31  # 31x31
arcsec=0.06*pix
for file in os.listdir('HST-Final'):
    fits_file = fits.open('HST-Final/'+file)
    data = fits_file[0].data
    header = fits_file[0].header
    w = wcs.WCS('HST-Final/'+file)
    print(w)
    ra_,dec_ = obsids[file.split('_')[0]]
    print(ra_,dec_)
    coo=SkyCoord(ra=ra_*u.degree,dec=dec_*u.degree,unit='deg',frame=FK5)
    #print(coo.to_pixel(wcs.WCS(header)))
    #print(skycoord_to_pixel(coo, w))
    d=Cutout2D(data,coo,u.Quantity((arcsec,arcsec),u.arcsec),wcs=w,mode="trim").data
    print(d)

WCS Keywords

Number of WCS axes: 2
CTYPE : ''  ''  
CRVAL : 0.0  0.0  
CRPIX : 0.0  0.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : 1.0  1.0  
NAXIS : 0  0
187.24380843 -0.71369079


ValueError: WCS should contain celestial component